# REINFORCE and AC

- 在前面的模型中，都假设给定state s，有最优策略，因此$\pi(a=a^*|s)=1, \pi(a\ne a^*|s)=0$。
  - 在两个action的价值相同时，策略可以任选其中一种，或者以某种概率选择两种action。

## I. 策略梯度的目标

### I.1 目标分析

#### I.1.1 理论目标

$$U(\theta) 
 = E[V(s)|\pi_{\theta }]=E_{s\sim d^{\pi }(s)}[V^{\pi }(s)]=\sum_{s\in \mathcal{S} }d(s)V^{\pi }(s)$$
  - <font color=brown>math of RL书中的展开方法</font>
  - 其中d(s)是状态s在当前策略下的稳定分布。这里假设该分布存在。
  - 但实践中通常不会等策略迭代无穷次达到稳定分布后再执行算法，所以真实环境下d(s)通常不会被考虑在迭代式中。

#### I.1.2 实际分析使用的目标

$$\begin{align}
U(\theta) = E_{\tau }R(\tau|\pi_{\theta}) = \sum_{\tau }P(\tau ;\theta )R(\tau )=\left\{\begin{matrix}
 E\left[ \sum_{t=0}^{H-1}R(s_t, u_t)|\pi_{\theta } \right]  & , undiscounted\ version\\
 E\left[ \sum_{t=0}^{H} \gamma^t R(s_t, u_t)|\pi_{\theta } \right] & , discounted\ version
\end{matrix}\right.
\end{align}$$

### I.2 梯度推导

#### I.2.1 梯度的估计量

$$\begin{align}
\nabla _{\theta }U(\theta ) 
& = \nabla _{\theta } \sum_{\tau }P(\tau ;\theta )R(\tau ) \\
& =  \sum_{\tau }\nabla _{\theta }P(\tau ;\theta )R(\tau ), {\color{green}{这步是因为\theta带给\tau的随机性都体现在P(\tau;\theta)里面}} \\
& =  \sum_{\tau }P(\tau ;\theta )\frac{\nabla _{\theta }P(\tau ;\theta )}{P(\tau ;\theta )}R(\tau ) \\
& =  E_{\tau }\nabla _{\theta }logP(\tau ;\theta )R(\tau ) \\
\\
\nabla _{\theta }logP(\tau ;\theta ) 
& = \nabla _{\theta }log\left[ d(s_0)\prod_{t=0}^{H-1} P(s_{t+1}|s_t, u_t)*\pi_{\theta }(u_t|s_t) \right ]\\
& = \nabla _{\theta }\left[logd(s_0) + \sum_{t=0}^{H-1}logP(s_{t+1}|s_t, u_t) + \sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\right ]\\
& = \underset{0}{ \underbrace{\nabla _{\theta }logd(s_0)}}  + \underset{0}{\underbrace{\nabla _{\theta }\sum_{t=0}^{H-1}logP(s_{t+1}|s_t, u_t)}} + \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\\
& = \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t) \\
\end{align}$$

$$\begin{align}
取梯度：g & = \nabla _{\theta }U(\theta )\\
& = E_{\tau }\left[\nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)R(\tau )\right ]\\
& = E\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\sum_{t=0}^HR(s_t, u_t)\right ]
\end{align}$$

- <font color=brown>没有用math of RL书中的展开方法，用的是Pieter Abbeel的foundations of deep RL中的展开方法。上式中和Pieter Abbeel原文分析中不同之处在于加上了$\nabla _{\theta }logd(s_0)$。这是因为原文中有一个默认的简化，也即假设MDP过程的稳定分布不受策略的影响。math of RL书中有详细分析，这里折衷，将该假设显式化，相当于math of RL书中$\bar v^0_{\pi}$的情况。接近Theorem 9.2，但改为undiscounted case。</font>

- 因此，可以得到梯度的估计量：
$$\begin{align}
\hat g 
& = \frac{1}{m}\sum_{i=1}^M\nabla _{\theta }logP(\tau^{(i)};\theta )R(\tau^{(i)})\\
& = \frac{1}{m}\sum_{i=1}^M \left[\nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\right]R(\tau^{(i)})\\
& = \frac{1}{m}\sum_{i=1}^M \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\right]\sum_{t=0}^{H-1}R(s_t^{(i)},u_t^{(i)})\\
& = \frac{1}{m}\sum_{i=1}^M \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\right]Q(s_t^{(i)},u_t^{(i)})\\
\end{align}$$

#### I.2.2 理解梯度估计量

$$\begin{align}
supervised\ learning中：maximize\ likelihood\\
\nabla_{\theta }J(\theta )&= \frac{1}{n}\sum_{i=1}^N \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t^{(i)}|s_t^{(i)})\right]\\
RL中：policy\ gradient\\
\nabla_{\theta }J(\theta )&=\frac{1}{n}\sum_{i=1}^N \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t^{(i)}|s_t^{(i)})\right]Q(s_t^{(i)},a_t^{(i)})
\end{align}$$

1. 在有监督学习中，比如imitation learning MLE提高的是样本中出现过的$(a_t|s_t)$ pair的概率。借此实现向样本学习。习得的$\pi_{\theta}(a_t|s_t)$尽可能与模仿对象的决策策略一样。
   - 有监督学习中，样本提供的$(a_t|s_t)$ pair中，给定state s，对应的action a就是学习的target，因为样本提供的action都       是‘好’action。
   - <font color=blue>所以MLE中的迭代式会increase the probabilities of all these sample actions under the corresponding states.</font>
2. 在策略梯度中，没有‘好的’目标样本可供学习。每次迭代时得到的$(a_t|s_t)$ pair都是用前一次迭代得到的策略与环境交互生成的，这些策略可能好，可能坏。所以迭代的目标应该是增加这些样本action中'好的'action的发生概率，减少‘坏的’action的发生概率。
   - <font color=blue>策略梯度算法中，决定这些action好坏的依据是这些action的rewards，也是它们对应的行为价值。直观理解，高价值的行为的概率被提高，低价值行为的概率被减少。 </font>
   - <font color=orange>但是这些行为的价值并不知道，只能估计。这里估计有两个限制，一个是只能用当前策略估计，而不是最优策略做估计；二是即使想知道当前策略下的行为价值，也需要用Monte Carlo或者Q-learning来估计。</font>

#### I.2.3 梯度的估计量的性质：无偏但方差大

- 无偏性： $E(\hat g) = \nabla_{\theta}U(\theta)$ [证略]
- 方差：因为用抽样法得到trajectory，在s和a的取值空间较大时，抽样结果的方差很大

## II. 最简单的Policy gradient算法

### II.1 迭代式

- <font color=blue>同样适用TD类的算法：</font>
    - 原问题：$max U(\theta) = max E_{\tau }R(\tau|\pi_{\theta})$
    - 转化问题：$\nabla_{\theta} U(\theta ) = 0  \Leftrightarrow  E\nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\sum_{t=0}^{H-1}R(s_t,u_t) = 0$
    - 迭代变量：$\hat g(\theta_k, \eta_k)  = \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(k)}|s_t^{(k)})\right]Q(s_t^{(k)},u_t^{(k)})$
      - <font color=red>符号说明：k是index of trajectory, t是单个trajectory上的time steps的index，这里SGD对应的一个样本是一个trajectory而不是一次time step</font>

- 迭代式：
$$\begin{align}
\theta_{k+1} 
& = \theta_k +\alpha_k \hat g(\theta_k, \eta_k)  \\
& = \theta_k +\alpha_k\left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(k)}|s_t^{(k)})Q(s_t^{(k)},u_t^{(k)})\right]\\
\end{align}$$

- <font color=blue>**Intuition**</font>：
  - 迭代算法工作方式：
    - 如果样本trajectory上的回报为正，算法会增加该trajectory发生的概率
    - 如果样本trajectory上的回报为负，算法会减少它发生的概率
  - <font color=red>问题</font>：
    - 如果reward是认为设定的函数，可能所有的trajectory的reward都是大于0的，只是有大小之分，那么上面的算法迭代更新的效率就会很低。因为即使一个trajectory A的reward小于平均值，迭代还是会提升它的发生概率，只有靠下次别的更优秀的trajectory的概率得到更大的提升才会平衡掉A带来的影响。
    - <font color=green>这也为后续算法优化方法中增加baseline提供了依据。增加baseline后，如果trajectory A的reward小于均值，那么扣掉baseline后得到的reward就是负值，就不存在上面的问题。</font>

### II.2 REINFORCE算法：MC policy gradient

- <font color=blue>**REINFORCE的迭代目标和估计式**：</font>
$$\begin{align}
g & = \underset{\tau\sim P_{\theta }(\tau )}{E}\left[\sum_{t=0}^{H-1} \nabla _{\theta }log\pi_{\theta }(a_t|s_t)\left(\sum_{t'=t}^{H-1}R(s_{t'}, a_{t'})-b\right)\right ]\\
\\
\hat g & = \frac{1}{N}\sum_{i=1}^N \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left(\sum_{t'=t}^{H-1}r(s_{t'}^{(i)}, a_{t'}^{(i)})-b\right)\right]\\
\end{align}$$

- 算法说明：
  - 迭代用SGD方法
  - Q(s,u)用MC抽样估计

- 算法伪码：<font color=brown>[ref: 动手学强化学习]</font>
  - 初始化参数$\theta$
  - 迭代M个trajectory：
    - 采样一个trajectory，轨迹是${s_1,a_1,r_1,s_2,a_2,r_2,...,s_T,a_T,r_T}$
    - 计算当前轨迹每个时刻t往后的回报：$\hat Q(s_t,u_t)=\sum_{j=t}^T\ \gamma^{j-t} r_{j}$
    - 按迭代式更新$\theta$

## III. 降低梯度估计量的方差

- 出发点：在MC policy gradient的基础上降低迭代所使用的梯度的方差
- 方法：
  1. 用discounted reward(上面算法中已经加上)
  2. 只计算'rewards to go'
  3. 增加baseline
  4. 把MC方法换成TD
- 得到的结果就是advantage actor critic算法

### III.1 rewards to go和baseline

#### III.1.1 对梯度估计量的影响：

1. rewards to go不改变原梯度期望值。<font color=brown>[证明见附页：spinning up的证明]</font>
$$\begin{align}
g & = E\left[ \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\sum_{t=0}^{H-1}R(s_t, u_t)\right ] \\
& = E\left[ \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\sum_{t'=t}^{H-1}R(s_{t'}, u_{t'})\right ]
\end{align}$$

2. 只要baseline的取值不取决于action，即使它的取值是state的函数也没关系，此时增加baseline不改变policy gradient估计量的无偏性。<font color=brown>[证明见math of RL page225 10.2.1]</font>

$$\begin{align}
g & = E\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\sum_{t=0}^{H-1}R(s_t, u_t)\right ] \\
& = E\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\sum_{t'=t}^{H-1}R(s_{t'}, u_{t'})\right ] \\
& = E\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\left( \sum_{t'=t}^{H-1}R(s_{t'}, u_{t'}) - b(s_t)\right )\right ]\\
\end{align}$$

3. 当baseline取$b(s)=V^{\pi}(s)$时，会减少估计量的方差。
   - <font color=red>注意，此时b的取值虽然与策略相关，但并不受t时刻action $a_t$的影响，所以无偏性仍然成立。</font>

#### III.1.2 定义Advantage

- 含义：t时刻的action $u_t$带来的reward比平均reward有多少相对优势。

$$A_t= \sum_{t'=t}^{H-1}R(s_{t'}^{(i)}, u_{t'}^{(i)}) - V^{\pi }(s_t^{(i)}) = Q(s_{t}^{(i)}, u_{t}^{(i)}) - V^{\pi }(s_t^{(i)})
$$

#### III.1.3 改造后的梯度估计量

$$\begin{align}
g & = E\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\left( \sum_{t'=t}^{H-1}R(s_{t'}, u_{t'}) - b(s_t)\right )\right ]\\
& = E\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\left( \sum_{t'=t}^{H-1}R(s_{t'}, u_{t'}) - V^{\pi }(s_t)\right )\right ]\\
& = E\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)A_t\right ]
\\
\\
\hat g 
& = \frac{1}{m}\sum_{i=1}^m\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left( \sum_{t'=t}^{H-1}R(s_{t'}^{(i)}, u_{t'}^{(i)}) - V^{\pi }(s_t^{(i)})\right )\right ] \\
& = \frac{1}{m}\sum_{i=1}^m\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left( Q(s_{t}^{(i)}, u_{t}^{(i)}) - V^{\pi }(s_t^{(i)})\right )\right ] \\
& = \frac{1}{m}\sum_{i=1}^m\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\hat A\right ]
\end{align}$$

### III.2 估计advantage

$$\begin{align}
\hat A=\hat Q^{\pi }(s_{t}^{(i)}, u_{t}^{(i)}) - \hat V^{\pi }(s_t^{(i)})
\end{align}$$

- 不同的方法和利弊：
  1. 用MC估计$\hat Q$，unbiased但variance大
  2. 【A2C算法】用TD估计，将上式转化为：$\hat A_t=r_{t+1} + V^{\pi}(s_{t+1}^{(i)}) -  V^{\pi }(s_t^{(i)})$ ，此时variance相对MC小很多，但是biased
  3. 【A3C算法】取上面两种情况的折衷：$\hat A_t=r_{t+1} + r_{t+2}+ r_{t+3}+ V^{\pi}(s_{t+3}^{(i)}) -  V^{\pi }(s_t^{(i)})$

### III.3 A2C(Advantage Actor-Critic)算法：policy gradient by TD + baseline

#### III.3.1 算法说明

- <font color=blue>**A2C的迭代目标和估计式**：</font>
$$\begin{align}
g & = \underset{\tau\sim P_{\theta }(\tau )}{E}\left[ \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t)\left(\sum_{t'=t}^{H-1}R(s_{t'}, a_{t'})-V^{\pi_{\theta }}(s_t)\right)\right ]\\
\\
\hat g & = \frac{1}{N}\sum_{i=1}^N \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left(\sum_{t'=t}^{H-1}r(s_{t'}^{(i)}, a_{t'}^{(i)})-\hat V^{\pi_{\theta }}(s_t^{(i)})\right)\right]\\
& = \frac{1}{N}\sum_{i=1}^N \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left[r(s_{t}^{(i)}, a_{t}^{(i)})+\hat V^{\pi_{\theta }}(s_{t+1}^{(i)})-\hat V^{\pi_{\theta }}(s_t^{(i)})\right]\right]\\
\end{align}$$

  - 迭代用SGD方法，增加了baseline（如果没有baseline就是AC算法）
  - Q(s,u)用TD估计，另外有一个network用于估计行为价值函数，参数w。
    - DQN中的迭代式：$\color{Blue} {w_{t+1} = w_t + \alpha_t\left [r_{t+1}+\gamma \hat{v}(s_{t+1,w_t})-\hat{v}(s_t,w_t)\right  ]\nabla_w \hat{v}(s_t,w_t)}$
    - 这里为了统一两个网络的更新方式，做一点变化：
$$\begin{align}
& 取\delta_t = r_{t+1}+\gamma \hat{v}(s_{t+1,w_t})-\hat{v}(s_t,w_t) \\
& w_{t+1} = w_t + \alpha_t\sum_{t=0}^{H-1}\delta_t\nabla_wV_w(s_t)\\
\end{align}$$

#### III.3.2 最基础的A2C：Batch A2C，每次处理一个batch的trajectory

- <font color=brown>[ref: spinning up]</font>
  - <img src='pics/batch_vanilla_pg.png' width='75%'>

#### III.3.3 每次处理一个trajectory

- <font color=brown>[ref: 动手学强化学习]</font>
- 算法伪码：
    - 初始化参数$\theta$
    - 迭代M个trajectory：
      1. 采样一个trajectory，轨迹是${s_1,a_1,r_1,s_2,a_2,r_2,...,s_T,a_T,r_T}$
      2. 为每一步计算$\delta_t = r_{t+1}+\gamma \hat{v}(s_{t+1,w_t})-\hat{v}(s_t,w_t)$
      3. 用min MSE(mean squared error)为目标更新状态价值网络参数：$ w_{t+1} = w_t + \alpha_w\sum_{t=0}^{H-1}\delta_t\nabla_wV_w(s_t)$
      4. 更新策略网络参数:$\theta_{t+1} = \theta_t + \alpha_{\theta}\sum_{t=0}^{H-1}\delta_t\nabla_{\theta}log\pi_{\theta}(u_t|s_t)$

## IV. AC算法在实践中的design choice

### IV.1 从off-line到online actor-critic

#### IV.1.1 基础的online AC：每次处理一个time-step 数据

- <font color=brown>[ref: cs285 slides]</font>
- 说明：
  1. 更新策略网络和状态价值网络参数的顺序不重要
  2. 状态价值网络用DNN，目标都是$min_w L(w) = \frac{1}{2}\sum_i\|\hat v(s_i)-y_i\|^2$
  3. <font color=orange>动手学强化学习版本和这里的区别，除了这里改成了on-line之外，还有：这里更新策略网络参数时用更新后的状态价值网络重新计算了一次advantage，而前者每次迭代只算一次advantage，用它同时更新两个网络。**这两种方式差异不大**</font>
- <font color=blue>**on-policy online算法伪码**</font>：
    - 初始化参数$\theta$
    - 迭代：
      1. 用当前策略采样一个time-step，$\{s_t,a_t,r_{t+1},s_{t+1}\}$
      2. 计算$\hat y_t = r_{t+1}+\gamma \hat{v}(s_{t+1,w_t})$
      3. 用min MSE(mean squared error)为目标更新状态价值网络参数：$ \nabla_w L(w) = (\hat v(s_t,w_t)-y_t)\nabla_w\hat v_w(s_t)$
      4. 计算$\hat A_t = r_{t+1}+\gamma \hat{v}(s_{t+1,w})-\hat{v}(s_t,w)$
      5. 更新策略网络参数:$\theta_{t+1} = \theta_t + \alpha_{\theta}\hat A_t\nabla_{\theta}log\pi_{\theta}(u_t|s_t)$

#### IV.1.2 synchronized和asynchronized parallel AC

- <img src='pics/parallel_ac.png' width='80%'>

- **synchronized parallel AC**
  - 思路：只用单个time-step数据的online AC的variance太大，一种改进方法是同时用N个simulator跑实验，每个跑一个trajectory。这样每个time-step可以收集N个数据构成一个batch，用这个batch作为on-line AC的数据。
  - 问题：这种方式的sample效率低，因为每个simulator的每次采样都都要等其他simulator完成后才能进行下一次采样。
  - <font color=green>优化：将采样方式改成异步并行</font>

- **asynchronized parallel AC**
  - 思路：多个simulator各自按照各自的节奏做实验，simulator的数量不一定为batch size N。所有simulator一共生成N个data就可以构成一个batch。这N个samples不一定是均匀分布在各个simulator上的。取到一个batch之后，算法做一次迭代，更新参数，然后新的参数一次性更新到所有的simulator。
  - 问题：由于simulator不用等待彼此的进度，也不用等待算法更新参数，所以很可能取了一个batch后，算法还没有完成一次迭代，simulator中又用旧参数生成了数据。直到算法迭代完后，simulator才会用新的参数继续做实验。这就导致算法下一次取到的batch中含有旧参数生成的samples。
  - 实践：只要样本不是too old，一般问题不大
  - <font color=green>优化：直接将算法改成off-policy的，这样样本的生成和是取用就没有参数同步的问题</font>

### IV.2 从on-policy到off-policy online AC

#### IV.2.1 迭代式决定AC是on-policy算法

- REINFORCE的梯度迭代目标和梯度估计量：
$$\begin{align}
g & = \underset{\tau\sim P_{\theta }(\tau )}{E}\left[\sum_{t=0}^{H-1} \nabla _{\theta }log\pi_{\theta }(a_t|s_t)\left(\sum_{t'=t}^{H-1}R(s_{t'}, a_{t'})-b\right)\right ]\\
\\
\hat g & = \frac{1}{N}\sum_{i=1}^N \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left(\sum_{t'=t}^{H-1}r(s_{t'}^{(i)}, a_{t'}^{(i)})-b\right)\right]\\
\end{align}$$

- A2C的梯度迭代目标和梯度估计量：
$$\begin{align}
g & = \underset{\tau\sim P_{\theta }(\tau )}{E}\left[ \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t)\left(\sum_{t'=t}^{H-1}R(s_{t'}, a_{t'})-V^{\pi_{\theta }}(s_t)\right)\right ]\\
& = \underset{\tau\sim P_{\theta }(\tau )}{E}\left[ \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t)[Q^{\pi_{\theta }}(s_{t}, a_{t})-V^{\pi_{\theta }}(s_t)]\right ]\\
\\
\hat g & = \frac{1}{N}\sum_{i=1}^N \left[\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t^{(i)}|s_t^{(i)})\left[r(s_{t}^{(i)}, a_{t}^{(i)})+\hat V^{\pi_{\theta }}(s_{t+1}^{(i)})-\hat V^{\pi_{\theta }}(s_t^{(i)})\right]\right]\\
online\ AC\ estimate:\\
\hat g & = \nabla _{\theta }log\pi_{\theta }(a_t^{(i)}|s_t^{(i)})\left[r(s_{t}^{(i)}, a_{t}^{(i)})+\hat V^{\pi_{\theta }}(s_{t+1}^{(i)})-\hat V^{\pi_{\theta }}(s_t^{(i)})\right]\\
\end{align}$$

#### IV.2.2 AC算法中决定on-policy特征的原因分析

- **online AC迭代式分析**：
  - 估计量式中$\nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t)和V^{\pi_{\theta }}(s)$都是由当前策略决定的。因此AC的梯度估计量需要on-policy算法.
  - <font color=deeppink>**严格地说，这里涉及算法的两个位置**</font>：  <font color=brown>[详见后文算法伪码]</font>
    1. 状态价值网络中，要用当前策略得到$a_t=\pi(a_t|s_t)$，然后通过emulator与环境交互得到的$r(s_t,a_t,s_{t+1})更新V^{\pi_{\theta }}(s)$
    2. 计算梯度估计量时，同样要用当前策略得到的$a_t来计算\nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t)$
  - 这两步中，用当前策略得到$a_t=\pi(a_t|s_t)$即使在off-policy中也可以直接用现有网络通过抽样来做。所以只要解决了上面第一点中的策略依赖性，那么第2点很好处理。因为它不涉及样本，从而不需要考虑样本是否来自当前策略或者旧策略。

- 用MSE为目标估状态价值函数和行为价值函数时，<font color=blue>**状态价值函数和行为价值函数从样本获取signal都是通过reward，但是估计量对样本的要求不同**：</font>
  1. 状态价值函数估计时，需要当前策略来得到$a_t$，且$s'$分布要用策略来决定，然后才能用state-action pair从环境获取reward。因此,<font color=green> $V(s_t)$的估计是on-policy的</font>。
     - $$\begin{align}
V^{\pi}(s) &= E_{a\sim \pi(s)}Q^{\pi }(s,a)\\
& = E_{a\sim \pi(s)}[r(s,a)+\gamma V^{\pi}(s')]\\
& = r(s,\pi(s))+\gamma E_{s'\sim P(s'|s,\pi(s))}V^{\pi}(s'),假设deterministic\ action\\
\hat V^{\pi_\theta }(s_t)  \overset{目标}{\longrightarrow}y_t & = r(s_t, \pi(s_t)) + \gamma \hat V^{\pi_\theta }(s_{t+1})\\
\end{align}$$
  2. 行为价值函数估计时，state-action pair已知，所以获取reward不需要当前策略，且下一步的状态由环境决定。也因此,<font color=green>  $Q(s_t,a_t)$的估计可以是off-policy的</font>。
     - $$\begin{align}
Q^{\pi}(s,a) & = r(s,a)+\gamma E_{s'\sim P(s'|s, a)}V^{\pi}(s') \\
& = r(s,a)+\gamma E_{s'\sim P(s'|s, a),a'\sim \pi(s')}Q^{\pi}(s',a')\\
\hat Q^{\pi_\theta }(s_t, a_t) \overset{目标}{\longrightarrow} y_t & = r(s_t, a_t, s_{t+1}) + \gamma \hat Q^{\pi_\theta }(s_{t+1}, a_{t+1})
\end{align}$$

#### IV.2.3 从on-policy调整到off-policy

- on-policy和off-policy算法伪码对比：
  - <img src='pics/on_and_off-policy_ac.png' width='95%'>

- <font color=red>**还有一个容易被忽略的问题**：</font>
  - 改成off-policy之后，$s_t\sim P_{\theta}(s)$的要求不满足，此时状态实际上都来自旧策略带来的分布。只是实践中这点通常不处理。
- 此外，上面右图中off-policy的第第5步在实践中一般不用advantage，而是直接用Q，所以第4步不用做。这样处理的代价是variance大。但是由于此时是off-policy的算法，会一次迭代用N个samples构成的batch，这样可以抵消一部分variance。

- 调整前后的梯度迭代式的估计量对比：
$$\begin{align}
\hat g_{on-policy} & =  \nabla _{\theta }log\pi_{\theta }(a_t^{(i)}|s_t^{(i)})\hat A(s_t,a_t)\\
& = \nabla _{\theta }log\pi_{\theta }(a_t^{(i)}|s_t^{(i)})\left[r(s_{t}^{(i)}, a_{t}^{(i)})+\gamma\hat V^{\pi_{\theta }}(s_{t+1}^{(i)})-\hat V^{\pi_{\theta }}(s_t^{(i)})\right]\\
\\
\hat g_{off-policy} & = \frac{1}{N}\sum_{i=1}^N\nabla _{\theta }log\pi_{\theta }(a_t^{(i)}|s_t^{(i)})\hat Q(s_t^{(i)},a_t^{(i)})\\
\end{align}$$

- off-policy AC的梯度迭代式：<font color=orange>这种算法的本质是用off-policy的方法估Q(s,a)，然后代入online AC</font>
$$\begin{align}
g & = \underset{\tau\sim P_{\theta }(\tau )}{E}\left[ \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_{t}, a_{t})\right ]\\
& = \underset{s_t\sim P_{\theta }(s )}{E}\left[\underset{a_t\sim \pi_{\theta }(a|s)}{E} \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_{t}, a_{t})\right ]\\
& = \underset{s_t\sim P_{\theta }(s )}{E}\left[\underset{a_t\sim \pi_{\theta }(a|s)}{E} \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t){\color{red}{\hat Q^{\pi_{\theta_{old}}}(s_{t}, a_{t})}}\right ]\\
实际用 & \approx \underset{\color{red}{s_t\sim P_{\theta_{old} }}(s )}{E}\left[\underset{a_t\sim \pi_{\theta }(a|s)}{E} \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(a_t|s_t){\color{red}{\hat Q^{\pi_{\theta_{old}}}(s_{t}, a_{t})}}\right ]\\
\end{align}$$

### IV.3 A3C到GAE：更好的bias-variance tradeoff

略